In [215]:
%matplotlib widget

In [17]:
#load library
import ipywidgets as widgets
import cv2
import numpy as np
import pandas as pd
from ipywebrtc import CameraStream, ImageRecorder, VideoRecorder

import tensorflow as tf
from tensorflow import keras 
import sklearn as sk
from sklearn.datasets import make_classification
import scikitplot as skplt
from sklearn.model_selection import train_test_split # Helps with organizing data for training
from sklearn.metrics import confusion_matrix # Helps present results as a confusion-matrix


### Parameters to tune and Library installation needed
1. install pandas on windows using command: pip install pandas
2. Install xlrd to read excell file using pandas using command : pip install xlrd
Here we shall use all the parameters and later all will be combined code

Note: for video as input... the directory you are working inside that we expect there will be directory called videos and inside that corresponding directory names and video should be there.(example working dir/Videos/102)


In [29]:
#parameters
training_data_path = 'dataset/subsystem_2/Dataset_subsystem_2.csv'
test_data_path = 'dataset/subsystem_1/Dataset_Subsystem_1.xlsx'
#Read the directory name
video_subdirecory_name = "videos"
#directory_names= ["102","159","294","441","564","576","609","666","711","723"]
directory_names= ["723"]
conditions = ["open_palm","open_dorsal","fist_palm","fist_dorsal","three_fingers_palm","three_fingers_dorsal"]
#conditions = ["open_palm"]
video_file_extensions = [".mp4",".webm"]

number_of_epoch = 50
#Note Input of training is all land mark and output column name is output_label which classified fro 0 to 5

### Process data from training and test file 
Here all feature columns (landmark position) are of type Int64 and out put column name renamed as output_label
Note: output_column has been assigned 0 to 5 assigned class
Class names: 
0 = open palm,
1 = fist palm,
2 = three_fingers palm,
3 = open dorsal,
4 = fist dorsal,
5 = three_fingers dorsal,

Another column gesture is assigned 0 and 1. This we shall use on convolution model to identify if the hand is palm side or dorsal side
0 = palm side
1 = dorsal side

In [30]:
#process data function. Look iteration skipped because iteration reduce performance
def processedData(data):
    #process output data by labeling multiclass 0 to 5 instead of one hot encoding
    processing_data = data
    processing_data.loc[processing_data.camera_facing_side.isin(['open']) & processing_data.gesture.isin(['palm']), 'camera_facing_side'] = 0
    processing_data.loc[processing_data.camera_facing_side.isin(['fist']) & processing_data.gesture.isin(['palm']), 'camera_facing_side'] = 1
    processing_data.loc[processing_data.camera_facing_side.isin(['three_fingers']) & processing_data.gesture.isin(['palm']), 'camera_facing_side'] = 2
    processing_data.loc[processing_data.camera_facing_side.isin(['open']) & processing_data.gesture.isin(['dorsal']), 'camera_facing_side'] = 3
    processing_data.loc[processing_data.camera_facing_side.isin(['fist']) & processing_data.gesture.isin(['dorsal']), 'camera_facing_side'] = 4
    processing_data.loc[processing_data.camera_facing_side.isin(['three_fingers']) & processing_data.gesture.isin(['dorsal']), 'camera_facing_side'] = 5

    # this will be used later to classify if hand is open or dorsal side fro raw image
    processing_data.loc[processing_data.gesture == 'palm', 'gesture'] = 0
    processing_data.loc[processing_data.gesture == 'dorsal', 'gesture'] = 1
    #Convert all column to integer type
    processing_data = processing_data.astype({"camera_facing_side": np.int64, "gesture": np.int64}) 
    #rename output column name as label
    processing_data=processing_data.rename(columns = {'camera_facing_side':'output_label'})
    return processing_data



### Training data
Here you can drop columns and get the columns for input and as well as y_train

In [31]:
#Load Training Data
#Load Training Dataset dropping unnecessary columns
training_data = pd.read_csv(training_data_path, sep="," , na_values='?' , dtype={'camera_facing_side': str,'gesture': str}).dropna().reset_index().drop(columns=['ID','frame'])
processed_train_data= processedData(training_data)
#X_train = processed_train_data.drop(columns=['output_label','gesture'])
#Y_train = processed_train_data['output_label']


In [32]:
print(processed_train_data.shape)

(62219, 83)


## Train prediction Model (Choose NN or Random forest?) not implemented
Train the model here and find the accuracy.Train set and validation set learned as 80% - 20%
use code below to get train and test set(Coppied from validation)
        trainI = np.random.choice(processed_train_data.shape[0], size = train_size, replace=False)
        trainIndex = songClassifyData.index.isin(trainI)
        
        train_dataset = processed_train_data.iloc[trainIndex]
        #rest of index will be used as validation dataset for the model
        validation_dataset = processed_train_data.iloc[~trainIndex]
        
        X_train= train_dataset.drop(columns=['output_label','gesture'])
        Y_train = train_dataset['output_label']
        
        X_validation= validation_dataset.drop(columns=['output_label','gesture'])
        Y_validation = validation_dataset['output_label']

Note: not implemented here: learn the model 

In [35]:
#train_dataset = processed_train_data.iloc[trainIndex]
#rest of index will be used as validation dataset for the model
#validation_dataset = processed_train_data.iloc[~trainIndex]

X_train= processed_train_data.drop(columns=['output_label','gesture'])
Y_train = processed_train_data['output_label']

X_validation= validation_dataset.drop(columns=['output_label','gesture'])
Y_validation = validation_dataset['output_label']

In [47]:
print(X_train.head)

<bound method NDFrame.head of        index  palm_root_x  palm_root_y  palm_thumb_1_x  palm_thumb_1_y  \
0          0          494          295             412             266   
1          1          493          294             410             266   
2          2          493          294             409             267   
3          3          492          294             407             267   
4          4          491          294             405             267   
...      ...          ...          ...             ...             ...   
62214  62214            0            0               0               0   
62215  62215            0            0               0               0   
62216  62216            0            0               0               0   
62217  62217            0            0               0               0   
62218  62218            0            0               0               0   

       palm_thumb_2_x  palm_thumb_2_y  palm_thumb_3_x  palm_thumb_3_y  \
0       

### Neural network with dense layers

Data preprocessing

In [49]:
X_train = X_train.drop(columns=['index'])

Creation of a new class to fix a bug

In [50]:
# this snippet will dissapear at some point in the future
# it exists because I found two bugs in KerasClassifier while writing
# this notebook. 
# Issues are raised in the tensorflow repo and this should be fixed soon(TM)

from tensorflow.keras.wrappers.scikit_learn import KerasClassifier 

class KerasClassifier_Patched(KerasClassifier):
    # bugfix: classifier doesn't declare that it is a classifier
    # in the Scikit learn API
    _estimator_type = "classifier"
    
    # bugfix: the current wrapper does not work with HotOne encoded
    # labels
    # this is only a fix in the specific case of this notebook,
    # not a general one
    def score(self, x, y, **kwargs):
        _, accuracy = self.model.evaluate(x,y, verbose=0, **kwargs)
        return accuracy

Creation of the model

In [68]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

def setupModel(): #very similar to the one from Lab 2
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Input(shape=(80)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(6))
    model.add(tf.keras.layers.Softmax())
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-4),
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=[tf.keras.metrics.CategoricalAccuracy()])
    return model

final_model = KerasClassifier_Patched(build_fn=setupModel,
                                      epochs=3,
                               batch_size=50,
                               verbose=1)

#Model training
#final_model.fit(X_train,Y_train)

### n-fold Cross validation
Here learn the model and calculate accuracy average. 
=> not implemented here: learn the model and find the accuracy for each epoch

In [71]:
# n-fold cross validation
np.random.seed(1)
# Divide train and validation set as 80% and 20%
number_of_rows = len(processed_train_data.index)
train_size = int ( (number_of_rows * 8) / 10)
accuracy = np.zeros(number_of_epoch)
final_model.fit(X_train,Y_train)
for i in range(number_of_epoch):
        #get 80% randomly choice of trainindex
        trainI = np.random.choice(processed_train_data.shape[0], size = train_size, replace=False)
        trainIndex = processed_train_data.index.isin(trainI)
        
        train_dataset = processed_train_data.iloc[trainIndex]
        #rest of index will be used as validation dataset for the model
        validation_dataset = processed_train_data.iloc[~trainIndex]
        
        X_train= train_dataset.drop(columns=['output_label','gesture','index'])
        Y_train = train_dataset['output_label']
        
        X_validation= validation_dataset.drop(columns=['output_label','gesture','index'])
        Y_validation = validation_dataset['output_label']
        print(X_validation.shape)
        print(Y_validation.shape)
        
        #learn the model and find the accuracy for each epoch
        accuracy[i] = final_model.score(X_validation,Y_validation)

# find the average of accuracy            
average_accuracy = np.mean(accuracy, axis=0) 

Train on 49775 samples
Epoch 1/3
49775/49775 [==============================] - 3s 69us/sample - loss: 4.6914 - categorical_accuracy: 0.9623
Epoch 2/3
49775/49775 [==============================] - 3s 65us/sample - loss: 0.0481 - categorical_accuracy: 0.9967
Epoch 3/3
49775/49775 [==============================] - 3s 63us/sample - loss: 0.0025 - categorical_accuracy: 0.9995
(12444, 80)
(12444,)


ValueError: You are passing a target array of shape (12444, 1) while using as loss `categorical_crossentropy`. `categorical_crossentropy` expects targets to be binary matrices (1s and 0s) of shape (samples, classes). If your targets are integer classes, you can convert them to the expected format via:
```
from keras.utils import to_categorical
y_binary = to_categorical(y_int)
```

Alternatively, you can use the loss function `sparse_categorical_crossentropy` instead, which does expect integer targets.

### Test dataset 
read the test dataset

In [76]:
#Load test dataset input
test_data = pd.read_excel(test_data_path, index_col=0, comment='#').dropna().reset_index() 
processed_test_data= processedData(test_data).drop(columns=['ID'])
#X_test = processed_test_data.drop(columns=['output_label','gesture','source','frame'])
##Y_test = processed_test_data['output_label']

### Function write predicted output, landmark to the image

In [78]:
#Function to write output class on video frame and draw circle and line
def WriteImageframe(image_frame, img_x_test_input_array, output_class):
        cv2.putText(image_frame, "Predicted " + str(output_class), (5, 50),cv2.FONT_HERSHEY_SIMPLEX, 2, (128,0,0), 2)

        #draw circle and line
        xposroot = img_x_test_input_array["palm_root_x"].iloc[ 0 ] 
        yposroot = img_x_test_input_array["palm_root_y"].iloc[ 0 ] 
        root_connection_number = 2
        thumb_iteration = 2
        other_finger_iteration = 12
        other_finger_sub_iteration = 3
        for indexval in range(0,80,2):
            xpos = img_x_test_input_array.iloc[ 0 , indexval ] 
            ypos = img_x_test_input_array.iloc[ 0 , indexval + 1 ] 
            cv2.circle(image_frame,(xpos, ypos), 3, (128,0,50),5)

            if thumb_iteration == 0 :
                if other_finger_sub_iteration == 0:
                    other_finger_sub_iteration = 3
                    if other_finger_iteration == 0:
                        thumb_iteration = 2
                        other_finger_iteration = 12
                        root_connection_number += 2
                        xposroot = img_x_test_input_array["dorsal_root_x"].iloc[ 0 ] 
                        yposroot = img_x_test_input_array["dorsal_root_y"].iloc[ 0 ] 
            if thumb_iteration > 0 and  indexval >= root_connection_number :
                xposth1 = img_x_test_input_array.iloc[ 0 , indexval ] 
                yposth1 = img_x_test_input_array.iloc[ 0 , indexval + 1 ] 
                xposth2 = img_x_test_input_array.iloc[ 0 , indexval + 2] 
                yposth2 = img_x_test_input_array.iloc[ 0 , indexval + 3] 
                thumb_iteration -= 1
                if thumb_iteration == 0 :
                    root_connection_number += 6
                if (xposth1 != 0 and yposth1 != 0):
                        if (xposth2 != 0 and yposth2 != 0):
                            cv2.line(image_frame, (xposth1, yposth1), (xposth2, yposth2), (128, 0, 10), 1)
                if indexval == root_connection_number:
                    if (xposroot != 0 and yposroot != 0):
                        if (xposth1 != 0 and yposth1 != 0):
                            cv2.line(image_frame, (xposroot, yposroot), (xposth1, yposth1), (128, 0, 10), 1)           
            else:
                if thumb_iteration <= 0 and other_finger_iteration > 0 and other_finger_sub_iteration > 0 and indexval >= root_connection_number:
                    xposth1 = img_x_test_input_array.iloc[ 0 , indexval ] 
                    yposth1 = img_x_test_input_array.iloc[ 0 , indexval + 1 ] 
                    xposth2 = img_x_test_input_array.iloc[ 0 , indexval + 2 ] 
                    yposth2 = img_x_test_input_array.iloc[ 0 , indexval + 3 ] 
                    other_finger_iteration -= 1
                    other_finger_sub_iteration -= 1
                    if other_finger_sub_iteration == 0 :
                        root_connection_number += 8
                    if (xposth1 != 0 and yposth1 != 0):
                            if (xposth2 != 0 and yposth2 != 0):
                                cv2.line(image_frame, (xposth1, yposth1), (xposth2, yposth2), (128, 0, 10), 1)
                    if indexval == root_connection_number:
                        if (xposroot != 0 and yposroot != 0):
                            if (xposth1 != 0 and yposth1 != 0):
                                cv2.line(image_frame, (xposroot, yposroot), (xposth1, yposth1), (128, 0, 10), 1)  
        return image_frame

### Get the input video for each frame :  predict and write output to the image


In [81]:
#Subsystem 1 : Landmark extraction. Get input landmark from image and feed it to the model
import os
current_working_directory = os.getcwd()
for directory_name in directory_names:
    for video_name in conditions:
        for extension in video_file_extensions:
            video_path =directory_name + "\\" + video_name 
            #print(current_working_directory + "\\" + video_path + extension)
            input_video = cv2.VideoCapture(r"" + video_subdirecory_name + "\\" + video_path + extension)
            output_file_name = r"" + video_subdirecory_name + "\\" + video_path  + "_predicted.mp4"
            backend = cv2.CAP_ANY
            fourcc_code = cv2.VideoWriter_fourcc(*"H264")
            fps = 24
            frame_size = (640, 480)
            output_video = cv2.VideoWriter(output_file_name, backend, fourcc_code, fps, frame_size)

            ret, frame = input_video.read()
            counter = 0
            while ret:
                ret, frame = input_video.read()
                if not ret:
                    continue
                print(counter)
                data_frame = processed_test_data.loc[processed_test_data.source.isin([directory_name + "/" + video_name  + extension]) & processed_test_data.frame.isin([counter])]
                
                #Input landmark - feed this to prediction model
                X_test_input = data_frame.drop(columns=['output_label','gesture','source','frame'])
                if X_test_input.empty == False :
                    #put output class to the image from predicted model. Now put as on the data
                    #X_test_output = data_frame['output_label']
                    #output_frame = WriteImageframe( frame, X_test_input ,str(X_test_output.iloc[ 0 ] ))
                    print(final_model.predict(X_test_input.drop(columns=['index'])))
                else:
                    output_frame = frame
                output_video.write(output_frame)
                counter += 1
            input_video.release()
            output_video.release()



### Specialization : Not yet implemented(Do not run) : Choose one specialization

### Specialization 1: CNN to identify hand and predict output(from given video file as input)

In [ ]:
#CNN Classification model generation
# Import of keras model and hidden layers for  convolutional network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Flatten
# Construction of model
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(120, 320, 1))) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10, activation='softmax'))
# Configures the model for training
model.compile(optimizer='adam', # Optimization routine, which tells the computer how to adjust the parameter values to minimize the loss function.
              loss='sparse_categorical_crossentropy', # Loss function, which tells us how bad our predictions are.
              metrics=['accuracy']) # List of metrics to be evaluated by the model during training and testing.
# Trains the model for a given number of epochs (iterations on a dataset) and validates it.
model.fit(X_train, Y_train, epochs=5, batch_size=64, verbose=2, validation_data=(X_test, Y_test))
# Save entire model to a HDF5 file
model.save('handrecognition_model.h5')
test_loss, test_acc = model.evaluate(X_test, Y_test)

print('Test accuracy: {:2.2f}%'.format(test_acc*100))

### Specialization 1: Automatically detect hand landmark and provide as input to Model (Random forest or NN)

In [ ]:
#take live video as input

In [ ]:
#cross validation

### Write predicted output to csv 
This will have 2 column: frame number and output
Read this file for the robot part